# Prueba de Integración modulos A, B y C del proycto BRAIN-CEMISID-V4

## Comportamiento deseado por parte de la integración

1. Se inyecta un evento en la Red Neuronal Sensorial, generando dos flujos de salida: uno destinado al comparador interno del módulo B y otro hacia el módulo C, específicamente a la sección de historias de vida.
2. El comparador interno emite una respuesta a la Red Neuronal Sensorial con el propósito de refrescar la información del BCE de las neuronas.
3. El módulo C, encargado de las historias de vida, proyecta dos salidas hacia el módulo B para actualizar la atención.
4. El módulo B, a su vez, remite una salida al módulo C, orientada a los episodios de vida, para renovar las historias de vida almacenadas.
5. El módulo B despacha un valor BCE al Agente Inteligente alojado en el módulo A.
6. Con la entrada proporcionada, el Agente Inteligente modifica su estado. Si no alcanza un estado terminal, el ciclo se repite con el ingreso de nuevos eventos.



### Importación de modulos

In [1]:
# Modulo A
from michael_agent.intelligent_agent import Intelligent_agent
from michael_agent.sensory_system import Sensory_system
# Modulo B
from phi_agent.mind.mind import Mind
# Modulo C
from memory.history import History

from michael_agent.generator import Generator
from phi_agent.utils.utils import bce_agent_to_mind_translator, get_temporal_memory
import time
from rich import print
# Escala de degrees
LEN_DEGREE = 4

In [2]:
# Instancias de las clases
sensory_system = Sensory_system()
intelligent_agent = Intelligent_agent()
memory = History()
event_generator = Generator()


# Patrones para inicializar
patterns = event_generator.gen_patterns()
init_memories = event_generator.gen_init_events()
arr_events=list(init_memories.values())

# Inicializar los patrones en el sensory_system y memoria
init_patterns = sensory_system.init_patterns(patterns)
memory.init_history(sensory_system.to_memory())

#Iniciar eventos en sensory_system y memoria
init_events=sensory_system.init_event(arr_events)
memory.init_history_events(init_events)

In [3]:
# Instancia de la clases
mind = Mind()

# Bucle de eventos
break_loop = False

while True:

    # Generación de evento aleatorio para los siete sentidos
    sensory_events = event_generator.gen_event()
    print()

    agent_bce = intelligent_agent.status()
    print("\t\t[yellow]Estado Agente Inteligente:\t\t", agent_bce)

    # entrada de eventos a la red neuronal de michael
    sensory_events_bce, sensory_events_ids, sensory_events_all  = sensory_system.set_event(sensory_events)

    print("Información de los eventos sensoriales:", sensory_events_all)
    # entrada de eventos a memoria
    memory.get_events(sensory_events_ids)

    # entrada de bce de la rn por sentido del comparador 1 de maria para actualizar los bce de las neuronas
    rn_bce_by_senses = bce_agent_to_mind_translator(bce_senses=sensory_events_bce)

    # entrada de memoria del comparador 1 de maria
    memory_stats = memory.get_stats()

    bce_winners, bce_modified, winners = mind.call_internal_comparator(
        agent_bce=agent_bce,
        bce_senses=rn_bce_by_senses,
        memory_stats=memory_stats
    )

    arr_bce_winners = sensory_system.get_bce_from_mind(bce_winners)

    print("BCE Ganadores:", bce_winners, winners)

    # update de la red neuronal
    sensory_system.update_neuron(arr_bce_winners)

    memories = memory.get_memory_sequences(params=bce_modified)

    life_episodes = memory.get_life_episodes()
    temporal_memory = get_temporal_memory(life_episodes)
    new_thoughts_by_factor = mind.update_attention(memories=memories, temporal_memory=temporal_memory)

    for factor in new_thoughts_by_factor:
        for memory_sequence in new_thoughts_by_factor[factor]:
            memory.handle_attention(factor, memory_sequence=memory_sequence)

    new_bce = mind.get_unified_bce()

    print("[green]BCE de entrada al Agente Inteligente:\t\t\t", new_bce)
    print("\t\t[yellow]Estado Agente Inteligente:\t\t", intelligent_agent.add_bce(new_bce))

    for factor in intelligent_agent.status().state():
        if factor[1] == intelligent_agent.length:
            print("[red]DEAD")
            break_loop = True
            break
    if break_loop:
        break

    time.sleep(5)
mind.conscious.stop()


starting consciousness

phi windows

Estado Agente Inteligente:               Bio:+0.0 Cul:+0.0 Emo:+0.0

{'biological': None, 'cultural': None, 'emotional': None}

Información de los eventos sensoriales:
[
    [22, Bio:+0.0 Cul:+0.5 Emo:+0.25, 'sight_event_011', 'sight_event_014', 'sight'],
    [23, Bio:+0.75 Cul:-0.25 Emo:-0.5, 'hearing_pattern_005_b', 'hearing_event_013', 'hearing'],
    [18, Bio:+0.25 Cul:+0.25 Emo:+0.0, 'smell_event_005', 'smell_event_010', 'smell'],
    [17, Bio:-0.5 Cul:-0.75 Emo:-0.25, 'taste_event_004', 'taste_event_011', 'taste'],
    [20, Bio:+0.5 Cul:+0.0 Emo:+0.0, 'touch_event_013', 'touch_event_010', 'touch'],
    [25, Bio:+0.5 Cul:+0.0 Emo:-0.5, 'body_pattern_009_b', 'body_event_005', 'body'],
    [21, Bio:+0.25 Cul:+0.25 Emo:+0.0, 'time_event_005', 'time_event_003', 'time']
]

...

2023-08-31 21:19:28 [info     ] Updating attention
2023-08-31 21:19:28 [info     ] Current Phis


{
    'biological': 'smell_event_010,smell_event_005,smell_event_012,smell_event_002',
    'cultural': 'taste_event_011,taste_event_004,taste_event_010',
    'emotional': 'hearing_event_013'
}

BCE de entrada al Agente Inteligente:                    Bio:+0.0 Cul:-0.5 Emo:-0.75

Estado Agente Inteligente:               Bio:+0.25 Cul:-0.75 Emo:-1.0

DEAD

In [ ]:
# Instancia de la clases
mind = Mind()

# Bucle de eventos
break_loop = False

while True:

    # Generación de evento aleatorio para los siete sentidos
    sensory_events = event_generator.gen_event()
    print()

    agent_bce = intelligent_agent.status()
    print("\t\t[yellow]Estado Agente Inteligente:\t\t", agent_bce)

    # entrada de eventos a la red neuronal de michael
    sensory_events_bce, sensory_events_ids, sensory_events_all  = sensory_system.set_event(sensory_events)

    print("Información de los eventos sensoriales:", sensory_events_all)
    # entrada de eventos a memoria
    memory.get_events(sensory_events_ids)

    # entrada de bce de la rn por sentido del comparador 1 de maria para actualizar los bce de las neuronas
    rn_bce_by_senses = bce_agent_to_mind_translator(bce_senses=sensory_events_bce)

    # entrada de memoria del comparador 1 de maria
    memory_stats = memory.get_stats()

    bce_winners, bce_modified, winners = mind.call_internal_comparator(
        agent_bce=agent_bce,
        bce_senses=rn_bce_by_senses,
        memory_stats=memory_stats
    )

    arr_bce_winners = sensory_system.get_bce_from_mind(bce_winners)

    print("BCE Ganadores:", bce_winners, winners)

    # update de la red neuronal
    sensory_system.update_neuron(arr_bce_winners)

    memories = memory.get_memory_sequences(params=bce_modified)

    life_episodes = memory.get_life_episodes()
    temporal_memory = get_temporal_memory(life_episodes)
    new_thoughts_by_factor = mind.update_attention(memories=memories, temporal_memory=temporal_memory)

    for factor in new_thoughts_by_factor:
        for memory_sequence in new_thoughts_by_factor[factor]:
            memory.handle_attention(factor, memory_sequence=memory_sequence)

    new_bce = mind.get_unified_bce()

    print("[green]BCE de entrada al Agente Inteligente:\t\t\t", new_bce)
    print("\t\t[yellow]Estado Agente Inteligente:\t\t", intelligent_agent.add_bce(new_bce))

    for factor in intelligent_agent.status().state():
        if factor[1] == intelligent_agent.length:
            print("[red]DEAD")
            break_loop = True
            break
    if break_loop:
        break

    time.sleep(5)
mind.conscious.stop()


In [4]:
mind.conscious.stop()
